# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import api_key

gmaps.configure(api_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
# Read the cities.csv file to Data Frame

cities_data = pd.read_csv("output_data/cities.csv", encoding = "utf-8")
cities_data.head()

,City_Data,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,puerto ayora,40,EC,1615763509,74,-0.7393,-90.3518,82.40,9.22
1,1,dikson,100,RU,1615763801,99,73.5069,80.5464,-25.73,11.36
2,2,georgetown,20,MY,1615763625,83,5.4112,100.3354,78.80,5.75
3,3,severo-kurilsk,0,RU,1615763802,77,50.6789,156.1250,33.57,4.36
4,4,ongwediva,94,NaN,1615763803,69,-17.7833,15.7667,77.00,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
humidity = cities_data["Humidity"].astype(float)
max_humidity = humidity.max()
locations = cities_data[["Lat", "Lng"]]

In [15]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating= False, max_intensity=max_humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
hotel_df = cities_data.loc[(cities_data["Max Temp"]> 70) & (cities_data["Max Temp"] < 80) & (cities_data["Cloudiness"] == 0), :]
hotel_df = hotel_df.dropna(how = "any")
hotel_df.reset_index(inplace=True)
del hotel_df["index"]
hotel_df.head()

,City_Data,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,24,belmonte,0,BR,1615763808,69,-15.8631,-38.8828,78.80,10.36
1,62,guaratinga,0,BR,1615763818,69,-16.5667,-39.5667,78.80,10.36
2,95,saint-philippe,0,RE,1615763331,78,-21.3585,55.7679,78.80,16.11
3,96,sao filipe,0,CV,1615763348,71,14.8961,-24.4956,71.13,4.54
4,97,caravelas,0,BR,1615763441,83,-17.7125,-39.2481,75.56,5.68


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotellist = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]["Lat"]
    lng = hotel_df.loc[i]["Lng"]

    params = {
        "location" : f"{lat}, {lng}",
        "radius" : 5000,
        "types" : "hotel",
        "key" : api_key  
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params = params)
    jsn = requested.json()
    try:
        hotellist.append(jsn["results"][0]["name"])
    except:
        hotellist.append("")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how = "any")
hotel_df.head()        

,City_Data,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,24,belmonte,0,BR,1615763808,69,-15.8631,-38.8828,78.80,10.36,
1,62,guaratinga,0,BR,1615763818,69,-16.5667,-39.5667,78.80,10.36,
2,95,saint-philippe,0,RE,1615763331,78,-21.3585,55.7679,78.80,16.11,
3,96,sao filipe,0,CV,1615763348,71,14.8961,-24.4956,71.13,4.54,
4,97,caravelas,0,BR,1615763441,83,-17.7125,-39.2481,75.56,5.68,


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))